# Jetpack Compose - ResourceBound Pattern - Podstawy

W tej aplikacji przyjrzymy się zastosowaniu biblioteki zastosowaniu wzorca `ResourceBound`, który ułatwia obsługę różnych stanów (ładowanie, błąd, sukces) w przypadku obsługi danych z zewnętrznych serwisów.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExNXRjMjNlbjJmYTNiMW45OW1iZmh2dnpzN2l1MmxsejB0eWlqNDNnMCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/6lIYVQa6srpVlsTU7u/giphy.gif" width="200" />

Klasa `Resource` jest przydatna, gdy chcemy przekazywać różne stany operacji do interfejsu użytkownika. Na przykład, gdy pobieramy dane z sieci, możemy używać `Resource` do informowania interfejsu użytkownika o stanie ładowania, sukcesie lub błędzie, a także dostarczyć odpowiednie dane i komunikaty w zależności od wyniku operacji. Jest to często stosowane w architekturze **MVVM** (Model-View-ViewModel), aby lepiej zarządzać stanem interfejsu użytkownika i przekazywać wyniki operacji do widoku.

W tej aplikacji również wykorzystamy  [**JSONPlaceholder**](https://jsonplaceholder.typicode.com/), tym razem z endpointem `comments`

Rozpocznijmy od dodania zależności
```kotlin
implementation ("androidx.lifecycle:lifecycle-viewmodel-compose:2.6.1")
implementation ("androidx.lifecycle:lifecycle-viewmodel-ktx:2.6.1")
implementation ("androidx.lifecycle:lifecycle-runtime-compose:2.6.1")

implementation ("com.squareup.retrofit2:retrofit:2.9.0")
implementation ("com.squareup.retrofit2:converter-gson:2.9.0")
implementation ("com.squareup.okhttp3:logging-interceptor:4.10.0")
```

Nasza aplikacja wymaga dostępu do internetu, aby go uzyskać musimy dodać deklarację uprawnienia w pliku konfiguracyjnym aplikacji, który nazywa się `AndroidManifest.xml`. To uprawnienie informuje system operacyjny Android, że aplikacja potrzebuje dostępu do internetu.

In [ ]:
<uses-permission android:name="android.permission.INTERNET"/>

Następnie dodajmy model danych

In [ ]:
data class CommentResponseItem(
    val body: String,
    val email: String,
    val id: Int,
    val name: String,
    val postId: Int
)

Następnie dodajmy interfejs

In [ ]:
interface PlaceholderApi {
    @GET("comments")
    suspend fun comments(): Response<List<CommentResponseItem>>
}

Zwróćmy uwagę na zastosowanie obiektu `Response`, jest to obiekt, który reprezentuje odpowiedź HTTP otrzymaną od serwera po wysłaniu żądania HTTP. Jest to ważny element przy pracy z `Retrofit`, ponieważ umożliwia analizę wyników operacji sieciowych.

- Reprezentacja odpowiedzi HTTP: `Response` zawiera wszystkie informacje otrzymane od serwera w odpowiedzi na żądanie HTTP. Obejmuje to kod stanu HTTP, nagłówki, dane i inne metadane.
- Kod stanu HTTP: Obiekt `Response` zawiera kod stanu HTTP, który informuje o wyniku żądania. Na przykład, kod stanu 200 oznacza sukces, a kody stanu 4xx i 5xx oznaczają błędy. Możesz użyć kodu stanu, aby określić, czy operacja zakończyła się sukcesem czy błędem.
- Dane odpowiedzi: `Response` może zawierać dane przesłane przez serwer. Odpowiedź może być w formacie JSON, XML, tekstu lub innym, w zależności od formatu danych przekazywanych między klientem a serwerem.
- Nagłówki HTTP: Obiekt `Response` może również zawierać nagłówki HTTP przekazane przez serwer. Nagłówki mogą zawierać różne metadane dotyczące odpowiedzi, takie jak typ treści, długość treści, dane uwierzytelniające itp.
- Przetwarzanie odpowiedzi: Za pomocą obiektów `Response` można przetwarzać odpowiedzi w celu wyodrębnienia potrzebnych danych lub informacji zwrotnych z serwera.
- Obsługa błędów: `Response` umożliwia obsługę błędów HTTP. Jeśli serwer zwraca błąd, można to wykryć na podstawie kodu stanu i podjąć odpowiednie kroki, takie jak wyświetlenie komunikatu o błędzie użytkownikowi.
- Sprawdzanie poprawności odpowiedzi: Przy użyciu `Response` można sprawdzać, czy odpowiedź jest poprawna i zawiera oczekiwane dane.
- Przykładowe operacje na obiekcie `Response` w `Retrofit` obejmują sprawdzanie kodu stanu za pomocą `response.isSuccessful()`, odczytywanie danych za pomocą `response.body()`, pobieranie nagłówków za pomocą `response.headers()`, a także obsługę błędów, jeśli odpowiedź jest niepoprawna.
- Obiekty typu `Response` są używane w celu skonkretyzowania i analizy wyników operacji sieciowych.

Dodajmy instancję `Retrofit` oraz repozytorium.

In [ ]:
object RetrofitInstance {
    val api: PlaceholderApi by lazy {
        Retrofit.Builder()
            .baseUrl("https://jsonplaceholder.typicode.com/")
            .addConverterFactory(GsonConverterFactory.create())
            .build()
            .create(PlaceholderApi::class.java)
    }
}

In [ ]:
class CommentRepository {
    private val api = RetrofitInstance.api

    suspend fun getComments() = api.comments()
}

Zanim przejdziemy do viewmodelu, dodajmy klasę `Resource`, która pozwoli nam reprezentować różne stany operacji.

In [ ]:
sealed class Resource<T> (
    val data: T? = null,
    val message: String? = null
){
    class Success<T>(data: T) : Resource<T>(data)
    class Error<T>(message: String, data: T? = null) : Resource<T>(data, message)
    class Loading<T> : Resource<T>()
}

- `sealed class Resource<T>`: Jest to deklaracja klasy `Resource`, która jest opakowaniem wyniku operacji. Parametr generyczny `T` oznacza typ danych, które będą przechowywane w obiekcie `Resource`.
- `val data: T? = null`: To jest pole `data`, które przechowuje wynik operacji. Może to być obiekt zawierający `dane` lub `null`, jeśli operacja nie zwróciła wyniku.
- `val message: String? = null`: To jest pole `message`, które może zawierać wiadomość lub komunikat związany z wynikiem operacji. Jest to przydatne do przechowywania informacji zwrotnych lub błędów.
- `class Success<T>(data: T) : Resource<T>(data)`: Jest to podklasa `Resource`, która reprezentuje sukces operacji. Przyjmuje dane (`data`) jako argument konstruktora.
- `class Error<T>(message: String, data: T? = null) : Resource<T>(data, message)`: Jest to podklasa `Resource`, która reprezentuje błąd operacji. Przyjmuje wiadomość błędu (`message`) i opcjonalnie dane związane z błędem (`data`) jako argumenty konstruktora.
- `class Loading<T> : Resource<T>()`: Jest to podklasa `Resource`, która reprezentuje stan ładowania. Nie przyjmuje żadnych danych. Jest używana do informowania interfejsu użytkownika, że operacja jest w trakcie wykonywania.

Następnie przejdźmy do implementacji viewmodel.

In [ ]:
class CommentsViewModel : ViewModel() {
    private val repository = CommentRepository()
    private var _comments: MutableStateFlow<Resource<List<CommentResponseItem>>> = MutableStateFlow(Resource.Loading())
    val comments: StateFlow<Resource<List<CommentResponseItem>>> = _comments

    init {
        getCommentsList()
    }

    private fun getCommentsList() = viewModelScope.launch {
        _comments.value = Resource.Loading()
        val response = repository.getComments()
        _comments.value = handleCommentsResponse(response)
    }

    private fun handleCommentsResponse(response: Response<List<CommentResponseItem>>)
            : Resource<List<CommentResponseItem>> {
        if (response.isSuccessful)
            response.body()?.let { return Resource.Success(it) }
        return Resource.Error(response.message())
    }
}

Funkcja `handleCommentsResponse` jest używana do przetwarzania odpowiedzi HTTP z serwera w celu utworzenia odpowiedniego obiektu `Resource`. W zależności od wyniku operacji sieciowej, funkcja ta zwraca `Resource.Success` z danymi komentarzy lub `Resource.Error` z komunikatem błędu. 

- `if (response.isSuccessful)`: To jest warunek sprawdzający, czy odpowiedź jest udana. `response.isSuccessful` zwraca `true`, jeśli kod stanu HTTP w odpowiedzi jest w zakresie 2xx, co oznacza sukces.
- `response.body()?.let { return Resource.Success(it) }`: Jeśli odpowiedź jest udana, to wykorzystywany jest operator `?.let`, który sprawdza, czy ciało odpowiedzi (`response.body()`) nie jest `null`, a następnie tworzy obiekt `Resource.Success` z danymi komentarzy i go zwraca. Ten obiekt oznacza sukces operacji i zawiera dane komentarzy.
- `return Resource.Error(response.message())`: Jeśli odpowiedź nie jest udana (kod stanu HTTP jest różny od 2xx), to jest tworzony obiekt `Resource.Error` z komunikatem błędu uzyskanym z odpowiedzi (`response.message()`). Ten obiekt oznacza błąd operacji.

Funkcja `getCommentsList()` jest odpowiedzialna za inicjowanie operacji pobierania listy komentarzy, ustawianie odpowiedniego stanu `_comments` (`Loading`), wykonanie żądania HTTP, przetworzenie odpowiedzi i zaktualizowanie stanu `_comments` w zależności od wyniku operacji.

- `_comments.value = Resource.Loading()`: Na początku funkcja ustawia stan `_comments` na `Resource.Loading()`, co oznacza, że operacja pobierania jest w trakcie wykonywania. Jest to sygnał dla interfejsu użytkownika, że aplikacja jest w trakcie ładowania danych.
- `val response = repository.getComments()`: Następnie funkcja wywołuje metodę `getComments()` na instancji `repository`, aby wysłać żądanie HTTP do serwera w celu pobrania listy komentarzy.
- `_comments.value = handleCommentsResponse(response)`: Po otrzymaniu odpowiedzi HTTP, funkcja przekazuje ten obiekt `Response` do funkcji `handleCommentsResponse(response)`. Ta funkcja przetwarza odpowiedź i zwraca odpowiedni obiekt `Resource`, który reprezentuje stan komentarzy. Wynik jest przypisywany do `_comments`, co aktualizuje stan komentarzy w `CommentsViewModel`.

Przejdźmy do komponentu reprezentującego ekran.

In [ ]:
@Composable
fun CommentsScreen(){
    val viewModel: CommentsViewModel = viewModel()

    val response by viewModel.comments.collectAsStateWithLifecycle()


    when (response) {
        is Resource.Success -> { response.data?.let { ShowList(comments = it) } }
        is Resource.Error -> {  }
        is Resource.Loading -> {  }
    }
}

`when (response) { ... }`: Jest to konstrukcja `when`, która w zależności od stanu response wyświetla odpowiedni widok. Możliwe stany to `Resource.Success`, `Resource.Error` i `Resource.Loading`.
- `is Resource.Success -> { response.data?.let { ShowList(comments = it) } }`: Jeśli stan `response` to `Resource.Success`, to wywoływana jest funkcja `ShowList` z listą komentarzy dostępną w `response.data`.
- `is Resource.Error -> { }`: Jeśli stan `response` to `Resource.Error`, to nie są podejmowane żadne działania. Tutaj można dodać obsługę błędów, np. wyświetlanie komunikatu o błędzie.
- `is Resource.Loading -> { }`: Jeśli stan `response` to `Resource.Loading`, to nie są podejmowane żadne działania. To może być miejsce na wyświetlenie wskaźnika ładowania lub innej informacji o trwającym procesie ładowania.

In [ ]:
@Composable
private fun ShowList(comments: List<CommentResponseItem>) {
    LazyColumn {
        items(comments) { comment ->
            Column {
                Text(
                    text = "NAME:\n" + comment.name,
                    Modifier.padding(4.dp)
                )
                Text(
                    text = "CONTENT:\n" + comment.body,
                    Modifier.padding(4.dp)
                )
                Text(
                    text = "POST ID:\n" + comment.postId.toString(),
                    Modifier.padding(4.dp)
                )
                Spacer(modifier = Modifier.padding(12.dp))
            }
        }
    }
}

Możemy przetestować aplikację.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExNXRjMjNlbjJmYTNiMW45OW1iZmh2dnpzN2l1MmxsejB0eWlqNDNnMCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/6lIYVQa6srpVlsTU7u/giphy.gif" width="200" />